In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.utils import to_categorical
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install tensorflow numpy matplotlib scikit-learn

In [ ]:
pip install opencv-python


In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [ ]:
df = pd.read_csv("/kaggle/input/fer2013/fer2013.csv")

In [ ]:
y = to_categorical(df['emotion'].values, num_classes=len(df['emotion'].unique()))
X_train_str, X_val_str, y_train, y_val = train_test_split(
    df['pixels'].values,
    y,
    test_size=0.2,
    stratify=df['emotion'].values,
    random_state=42
)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input

rot_layer = tf.keras.layers.RandomRotation(factor=0.06)  

def parse_row(pixels, label, augment=False):
    pixels = tf.strings.split(pixels)
    pixels = tf.strings.to_number(pixels, out_type=tf.float32)
    pixels = tf.reshape(pixels, (48, 48, 1))

    img = tf.image.resize(pixels, (224, 224))
    img = tf.image.grayscale_to_rgb(img)  

    if augment:
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_brightness(img, max_delta=0.25)
        img = tf.image.random_contrast(img, lower=0.75, upper=1.25)
        img = tf.image.random_saturation(img, lower=0.75, upper=1.25)
        img = rot_layer(img)
        scale = tf.random.uniform([], 0.85, 1.0)
        new_size = tf.cast(scale * 224, tf.int32)
        img = tf.image.random_crop(img, size=[new_size, new_size, 3])
        img = tf.image.resize(img, (224, 224))

        if tf.random.uniform([]) > 0.7:
            erase_w = tf.random.uniform([], 30, 60, dtype=tf.int32)
            erase_h = tf.random.uniform([], 30, 60, dtype=tf.int32)
            x = tf.random.uniform([], 0, 224 - erase_w, dtype=tf.int32)
            y = tf.random.uniform([], 0, 224 - erase_h, dtype=tf.int32)
            mask = tf.ones((erase_h, erase_w, 3))
            paddings = [[y, 224 - erase_h - y], [x, 224 - erase_w - x], [0, 0]]
            mask = tf.pad(mask, paddings, constant_values=0)
            img = img * (1 - mask)

    img = preprocess_input(img)
    return img, label


batch_size = 16

train_ds = (
    tf.data.Dataset.from_tensor_slices((X_train_str, y_train))
    .map(lambda x, y: parse_row(x, y, augment=True), num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(2000)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

val_ds = (
    tf.data.Dataset.from_tensor_slices((X_val_str, y_val))
    .map(lambda x, y: parse_row(x, y, augment=False), num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)


In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # Phase 1: freeze base

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
output = Dense(y.shape[1], activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

In [ ]:
callbacks = [
    ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_accuracy", mode="max"),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
]

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

classes = np.arange(y.shape[1])  # [0..6]
y_integers = np.argmax(y_train, axis=1)

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y_integers
)

class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)


In [ ]:
model.compile(
    optimizer=Adam(1e-3), 
    loss="categorical_crossentropy",
    metrics=["accuracy"])
print("Phase 1 Training ...")
history_phase1 = model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=5, 
    callbacks=callbacks)

In [ ]:
model.save("/kaggle/working/emotion_model_phase1.h5")


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/kaggle/working/emotion_model_phase1.h5")

for layer in model.layers[-50:]:
    layer.trainable = True

model.compile(optimizer=Adam(1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

history_aug = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    class_weight=class_weights,
    callbacks=callbacks
)


In [ ]:
model.save("/kaggle/working/emotion_model_phase2.h5")



In [ ]:
import os

save_dir = "/kaggle/outputs/models"
os.makedirs(save_dir, exist_ok=True)

model.save(f"{save_dir}/emotion_model_phase2.h5")


In [ ]:
!cp /kaggle/working/emotion_model_phase2.h5 /kaggle/outputs/


In [ ]:
import os
print(os.listdir("/kaggle/working"))


In [ ]:
import json

with open("/kaggle/working/history_phase1.json", "w") as f:
    json.dump(history_phase1.history, f)

In [ ]:
import json
with open("/kaggle/working/history_phase2.json", "w") as f:
    json.dump(history_aug.history, f)

In [ ]:
import os
print(os.path.getsize("/kaggle/working/history_phase1.json"))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model

def plot_history(history, title):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history["accuracy"], label="Train Acc")
    plt.plot(history["val_accuracy"], label="Val Acc")
    plt.title(f"{title} - Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history["loss"], label="Train Loss")
    plt.plot(history["val_loss"], label="Val Loss")
    plt.title(f"{title} - Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    plt.show()

plot_history(history_phase1.history, "Phase 1 (Frozen Base)")

plot_history(history_aug.history, "Phase 2 (Fine-tuning)")

best_model = load_model("/kaggle/working/emotion_model_phase2.h5")

y_pred = np.argmax(best_model.predict(val_ds), axis=1)
y_true = np.argmax(y_val, axis=1)

print(" Classification Report:")
print(classification_report(y_true, y_pred, digits=4))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm, annot=True, fmt="d", cmap="Blues",
    xticklabels=range(len(df['emotion'].unique())),
    yticklabels=range(len(df['emotion'].unique()))
)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

model = load_model("/kaggle/working/emotion_model_phase2.h5")

for layer in model.layers:
    layer.trainable = True

In [ ]:
loss_fn = CategoricalCrossentropy(label_smoothing=0.1)
optimizer = AdamW(learning_rate=1e-5, weight_decay=1e-6)

model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=["accuracy"]
)
callbacks_phase3 = [
    EarlyStopping(
        monitor="val_loss",
        patience=6,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=2,
        min_lr=1e-7
    ),
    ModelCheckpoint(
        f"{save_dir}/emotion_model_phase3_best.h5",
        monitor="val_accuracy",
        save_best_only=True,
        mode="max"
    )
]

In [ ]:
history_phase3 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    class_weight=class_weights,
    callbacks=callbacks_phase3
)

In [ ]:
model.save("/kaggle/working/emotion_model_phase3.h5")

In [ ]:
import json
with open("/kaggle/working/history_phase3.json", "w") as f:
    json.dump(history_phase3.history, f)

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json

best_model = load_model("/kaggle/working/emotion_model_phase3.h5")

y_true = np.argmax(y_val, axis=1)
y_pred = np.argmax(best_model.predict(val_ds), axis=1)


print("Classification Report:")
print(classification_report(y_true, y_pred, digits=4))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=range(len(np.unique(y_true))),
            yticklabels=range(len(np.unique(y_true))))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix - Phase 3")
plt.show()

# ---------------------
# Load Training History (JSON format)
# ---------------------
with open("/kaggle/working/history_phase3.json", "r") as f:
    history_phase3 = json.load(f)


plt.figure(figsize=(14,5))

plt.subplot(1,2,1)
plt.plot(history_phase3["accuracy"], label="Train Acc")
plt.plot(history_phase3["val_accuracy"], label="Val Acc")
plt.title("Accuracy - Phase 3")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history_phase3["loss"], label="Train Loss")
plt.plot(history_phase3["val_loss"], label="Val Loss")
plt.title("Loss - Phase 3")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.show()


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np, tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

model = load_model("/kaggle/working/emotion_model_phase3.h5")  

def flip_map(img, y):
    return tf.image.flip_left_right(img), y

val_ds_flip = val_ds.map(flip_map, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

p1 = model.predict(val_ds, verbose=1)
p2 = model.predict(val_ds_flip, verbose=1)
p  = (p1 + p2) / 2.0
y_pred = p.argmax(axis=1)

ys = []
for _, y in val_ds:
    ys.append(y.numpy())
y_true = np.argmax(np.concatenate(ys, axis=0), axis=1)

print("TTA accuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))


In [ ]:
model.save("/kaggle/working/emotion_model_phase3_final.h5")


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("emotion_model_phase3_final.h5")


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/kaggle/working/emotion_model_phase3_final.h5")
model.summary()
